In [1]:
# 1: 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 2: 실습용 github 코드 클론
import os
if os.path.isdir('/content/drive/MyDrive/mygpt-lecture') is False: # 실습 코드가 없는 경우에만 클론
    %cd /content/drive/MyDrive/
    !git clone https://github.com/oglee815/mygpt-lecture.git
else:
    print("Already exist")

Already exist


# Stage 1. 도메인 특화 사전학습 데이터를 만들어보자

### Domain Adaptive Pretraining 데이터 만들기 (100% 수작업)

In [4]:
# 2: CSV로 만든 Pre-training 데이터 읽기
# 본인이 직접 만든 데이터를 읽어오세요!!
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/mygpt-lecture/data/stage1. domain_adaptive_pretraining/아이브.csv")

In [8]:
# 3: 데이터 살펴보기
data.head()

,주제,내용,길이
0,아이브의 개요,2021년 12월 1일에 데뷔한 스타쉽엔터테인먼트 소속의 대한민국 6인조 다국적 걸...,138
1,아이브의 멤버,"안유진, 가을, 레이, 장원영, 리즈, 이서",24
2,아이브의 콘셉트1,청순 콘셉트를 지나 다시금 걸 크러시 콘셉트가 트렌드로 돌아온 4세대 걸그룹답게 I...,405
3,아이브의 콘셉트2,사실 이 차별화는 현재 걸크러시 콘셉트로 활동하는 걸그룹 대부분에게 해당되는 말이긴...,380
4,아이브의 콘셉트3,연타석 홈런을 친 데뷔 싱글 1집 타이틀곡 <ELEVEN>과 싱글 2집 타이틀곡 <...,269


In [9]:
# 3: 데이터 갯수 찍어보기
print(len(data))

25


In [10]:
# 4: 데이터 전처리

# 주제만 가지고 학습을 위한 Prompt 형식으로 변환합니다.
def prompting(topic):
    return f"다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:{topic}\n"

# 학습에 사용되는 포맷인 jsonl 타입으로 변경합니다
instruction = []
for a in data.iterrows():
    topic = a[1]['주제']
    output = a[1]['내용']
    instruction.append({'instruction': prompting(topic), 'output': output})

In [ ]:
# 5: 알맞은 경로에 알맞은 이름으로 데이터 저장
with open("/content/drive/MyDrive/mygpt-lecture/data/stage1. domain_adaptive_pretraining/domain_adaptive_ive_pretrain.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in instruction: file.write(json.dumps(i, ensure_ascii=False) + "\n")

# API test

In [16]:
# 6: 라이브러리 설치
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [36]:
# 7: ChatGPT API 테스트
import openai

openai.api_version=""
openai.api_key="Your Key"
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
      messages=[
        {"role": "user", "content": "아이브는 어떤 소속사에 소속되어 있나요?"}],
    max_tokens=512,
    temperature=0,
)

print(response['choices'][0]['message']['content'])

아이브는 SM 엔터테인먼트 소속입니다.


# Stage1 데이터 생성하기, ChatGPT를 활용하여 도메인 특화 질의 응답 생성



In [12]:
# 8: ChatGPT API 테스트

# 자신이 만든 데이터 읽어오기!!
import json

data=[]
for a in open("/content/drive/MyDrive/mygpt-lecture/data/stage1. domain_adaptive_pretraining/domain_adaptive_ive_pretrain.jsonl", 'r'):
    data.append(json.loads(a))

print(data[0]['instruction'])
print(data[0]['output'])

다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.
주제:아이브의 개요

2021년 12월 1일에 데뷔한 스타쉽엔터테인먼트 소속의 대한민국 6인조 다국적 걸그룹이다.[2]

그룹명인 IVE는 'I HAVE'의 축약형인 'I'VE'에서 유래되었으며, "우리가 가진 것들을 모두 당당히 보여 드리겠다."는 포부를 담고 있다.


In [13]:
# 9: ChatGPT에게 보낼 Prompt 설정

# ChatGPT에게 여러번 물어보면 그만큼 돈과 시간이 많이 들겠죠?
# 따라서 10개의 질문/정답 쌍을 한번에 받습니다!

PROMPT="다음 주제와 내용을 보고 답변 할 수 있는 질문과 정답 쌍을 10개 만들어줘.\n주제:{title}\n내용:{content}\n\
답변은 반드시 아래 형태를 띄어야 해.\n\
Q1:질문\n\
A1:정답\n\
\n\
Q2:질문\n\
A2:정답\n\
\n\
Q3:질문\n\
A3:정답\n\
\n\
Q4:질문\n\
A4:정답\n\
\n\
Q5:질문\n\
A5:정답\n\
...\n\
Q10:질문\n\
A10:정답\n\
"

In [14]:
# 10: 데이터 담을 리스트 생성
qa_data = []
error_data = []

In [21]:
# 11: API Call

# ChatGPT API를 통해 데이터 얻어오기
# 중간에 에러가 나면 error_data에 넣어두고, 아래 for_loop의 data -> error_data 로 고쳐서 다시 돌려도 됨!!
# 데이터에 민감 정보가 있을 경우 에러가 날 수 있습니다!!

from tqdm import tqdm

for a in tqdm(data[:10]):
    title = a['instruction'].split("주제:")[-1]
    content = a['output']
    try:
        response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[
                {'role':"system","content":"You are a helpful assistant."},
                {'role':"user","content":PROMPT.format_map({'title':title, 'content':content})},
            ]
        )

        qa_data.append(response['choices'][0]['message']['content'])

    except Exception as e:
        error_data.append(a)
        print(title)
        print(e)

100%|██████████| 10/10 [10:14<00:00, 61.45s/it]


In [24]:
# 12: 학습이 시작되는 동안 직접 데이터를 만들어 볼까요?
hand_written_data = [
    {'instruction': "아이브의 멤버 중 반민초파는 누구이고, 반민초파라는 단어의 의미는 무엇입니까?",
    'output' : "아이브의 멤버 중 반민초파는 가을, 리즈로 알려져 있습니다. 반민초파라는 단어의 의미는 민초가 들어간 음식을 좋아하지 않는다는 의미입니다."
    },
    {'instruction': "",
    'output' : ""
    },
    {'instruction': "",
    'output' : ""
    }
    ### 이어서 작성
]

In [25]:
# 13: chatgpt의 응답에서 Q와 A를 발라냅니다.
instruction_data=[]
for a in qa_data:
    try:
        for i, b in enumerate(a.split("\n\n")):
            i +=1
            q = b.split("Q"+str(i)+":")[1].split("\nA"+str(i)+":")[0].lstrip()
            ans = b.split("A"+str(i)+":")[1].split("\n\nQ"+str(i)+":")[0].lstrip()
            instruction_data.append({'instruction':q,'output':ans})
    except Exception as e:
        print(e)
        continue

In [ ]:
len(instruction_data)

248

In [ ]:
# 14: 잘 만들어졌는지 볼까요?
for a in instruction_data:
    print(a)

{'instruction': '아이브는 어떤 소속사에 소속되어 있나요?', 'output': '아이브는 스타쉽엔터테인먼트에 소속되어 있습니다.'}
{'instruction': '아이브는 몇 명으로 이루어져 있나요?', 'output': '아이브는 6인조로 이루어져 있습니다.'}
{'instruction': '아이브의 데뷔일은 언제인가요?', 'output': '아이브의 데뷔일은 2021년 12월 1일입니다.'}
{'instruction': 'IVE의 그룹명은 어떤 의미를 가지고 있나요?', 'output': 'IVE는 \'I HAVE\'의 축약형인 \'I\'VE\'에서 유래되었으며, "우리가 가진 것들을 모두 당당히 보여 드리겠다."는 포부를 담고 있습니다.'}
{'instruction': '아이브는 어떤 국적을 가진 걸그룹인가요?', 'output': '아이브는 다국적 걸그룹입니다.'}
{'instruction': '아이브의 소속사는 어떤 엔터테인먼트 기업인가요?', 'output': '아이브의 소속사는 스타쉽엔터테인먼트입니다.'}
{'instruction': '아이브의 데뷔 후 첫 음악 프로그램 1위는 언제였나요?', 'output': '아이브의 데뷔 후 첫 음악 프로그램 1위는 공개되지 않았습니다.'}
{'instruction': '아이브의 멤버는 누구들인가요?', 'output': '아이브의 멤버는 공개되지 않았습니다.'}
{'instruction': '아이브의 음악 스타일은 어떤 특징이 있나요?', 'output': '아이브의 음악 스타일은 아직 공개되지 않았습니다.'}
{'instruction': '아이브는 데뷔 후 어떤 활동을 할 예정인가요?', 'output': '아이브의 데뷔 후 활동 계획은 아직 공개되지 않았습니다.'}
{'instruction': '아이브의 멤버는 누구인가요?', 'output': '안유진, 가을, 레이, 장원영, 리즈, 이서'}
{'instruction': '안유진은 어떤 역할을 맡고 있나요?', 'output': '안유진

In [27]:
# 15: 직접 만든 데이터와 ChatGPT가 만든 데이터를 합칩니다.
instruction_data.extend(hand_written_data)

In [ ]:
# 16: Train/Test로 적당히 나눕니다.
import random
test_data=[]
train_data=[]

for a in instruction_data:
    if random.random() < 0.1:
        test_data.append(a)
    else:
        train_data.append(a)

In [ ]:
# 17: 알맞은 이름으로 저장
import json
with open("/content/drive/MyDrive/mygpt-lecture/data/stage1. domain_instruction_tuning/ive_instruction_train.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in train_data: file.write(json.dumps(i , ensure_ascii=False) + "\n")

with open("/content/drive/MyDrive/mygpt-lecture/data/stage1. domain_instruction_tuning/ive_instruction_test.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in test_data: file.write(json.dumps(i, ensure_ascii=False) + "\n")

# Stage 2의 RM 데이터 생성

### Data Format

data = [{'prompt':"질문", 'chosen':"더 우수한 답변", 'reject':"덜 우수한 답변"}]

### 좋은 답변 만들기

위에서 이미 만들어진 만든 데이터를 사용합니다

In [47]:
# 18: 위에서 만든 instruction 데이터 읽어오기!!

instruction_data=[]
for a in open("/content/drive/MyDrive/mygpt-lecture/data/stage1. domain_instruction_tuning/ive_instruction_train.jsonl", 'r'):
    instruction_data.append(json.loads(a))

### 덜 좋은 답변 만들기

참고자료를 주지 않고, 오로지 상상에만 맡겨서 데이터를 만들기!

그럼 자연스럽게 퀄러티가 떨어지는 답변을 얻을 수 있습니다!

ChatGPT는 아이브에 대해 알지 못하니까요^^

In [54]:
# 19: 일단 위에서 만든 데이터에서 질문만 뽑아보자
questions = [a['instruction'] for a in instruction_data]
print("질문 예시:", questions[0])
print("질문 갯수:", len(questions))

질문 예시: 아이브는 어떤 소속사에 소속되어 있나요?
질문 갯수: 223


In [90]:
# 20: 질문을 10개 단위로 쪼개기
question_list = ["Q" + str((i%10)+1)+":" + q  for i, q in enumerate(questions)]

question_set =[]
for i in range(0, len(question_list), 10):
    question_set.append("\n".join(question_list[i:i+10]))

print("전체 질문 Set 갯수: ", len(question_set))
print("질문 Set 예시:\n", question_set[0])

전체 질문 Set 갯수:  23
질문 Set 예시:
 Q1:아이브는 어떤 소속사에 소속되어 있나요?
Q2:아이브는 몇 명으로 이루어져 있나요?
Q3:아이브의 데뷔일은 언제인가요?
Q4:IVE의 그룹명은 어떤 의미를 가지고 있나요?
Q5:아이브는 어떤 국적을 가진 걸그룹인가요?
Q6:아이브의 데뷔 후 첫 음악 프로그램 1위는 언제였나요?
Q7:아이브의 멤버는 누구들인가요?
Q8:아이브의 음악 스타일은 어떤 특징이 있나요?
Q9:아이브는 데뷔 후 어떤 활동을 할 예정인가요?
Q10:아이브의 멤버는 누구인가요?


In [91]:
# 21: ChatGPT에게 물어볼 프롬프트 형식
PROMPT="### Instruction: 주어진 질문들에 대해서 각각 사실대로 답변해줘. 각 답변은 아래와 같은 형식으로 답변해줘.\n\
\n\
### 답변 형식\n\
A1:답변1\n\
A2:답변2\n\
...\n\
A10:답변10\n\n\
### 질문들\n\
{questions}\n"

print(PROMPT.format_map({'questions':question_set[0]}))

### Instruction: 주어진 질문들에 대해서 각각 사실대로 답변해줘. 각 답변은 아래와 같은 형식으로 답변해줘.

### 답변 형식
A1:답변1
A2:답변2
...
A10:답변10

### 질문들
Q1:아이브는 어떤 소속사에 소속되어 있나요?
Q2:아이브는 몇 명으로 이루어져 있나요?
Q3:아이브의 데뷔일은 언제인가요?
Q4:IVE의 그룹명은 어떤 의미를 가지고 있나요?
Q5:아이브는 어떤 국적을 가진 걸그룹인가요?
Q6:아이브의 데뷔 후 첫 음악 프로그램 1위는 언제였나요?
Q7:아이브의 멤버는 누구들인가요?
Q8:아이브의 음악 스타일은 어떤 특징이 있나요?
Q9:아이브는 데뷔 후 어떤 활동을 할 예정인가요?
Q10:아이브의 멤버는 누구인가요?



In [92]:
# 22: 데이터를 담을 리스트
bad_qa=[]
bad_qa_error=[]

In [94]:
# 23: ChatGPT API Call
# 에러가 발생하면 에러가 난 데이터만 다시 시도

from tqdm import tqdm
import time

for a in tqdm(question_set[:2]):
    try:
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=[
                {'role':"system","content":"You are a helpful assistant."},
                {'role':"user","content": PROMPT.format_map({'questions':a})},
                ],
            temperature=1,
        )
        bad_qa.append({'instruction': a,  'output':response['choices'][0]['message']['content']})
        time.sleep(1) # Optional
    except Exception as e:
        bad_qa_error.append(a)
        print(title)
        print(e)

100%|██████████| 2/2 [00:46<00:00, 23.17s/it]


In [ ]:
# 24: 데이터가 만들어지는 동안 직접 만들어보기!
hand_written_dataset=[
    {'prompt': '아이브와 아이유의 차이는 무엇인가요?',
    'chosen': '아이브는 2021년 12월 1일에 데뷔한 스타쉽엔터테인먼트 소속의 대한민국 6인조 다국적 걸그룹입니다. 아이유는 개인으로 활동하는 대한민국의 싱어송라이터 입니다.',
     'rejected': '아이브와 아이유는 아씨 성을 가진 자매입니다.'},
    {'prompt': '질문',
    'chosen': '좋은 답변',
     'rejected': '나쁜 답변'}
    #...
]

In [97]:
print(bad_qa[0]['output'])

A1: 아이브는 스타제국 에서 소속되어 있어요.
A2: 아이브는 5명으로 이루어져 있어요.
A3: 아이브의 데뷔일은 2021년 12월 14일이에요.
A4: IVE는 "Inborn Voice"의 줄임말로 탄생한 목소리라는 의미를 가지고 있어요.
A5: 아이브는 대한민국의 걸그룹이에요.
A6: 아이브의 데뷔 후 첫 음악 프로그램 1위는 미정입니다.
A7: 아이브의 멤버는 다음과 같아요: 태연, 보라, 장미, 디은, 저린
A8: 아이브의 음악 스타일은 다양한 장르를 탐구하며 전 세계적인 음악적 트렌드를 수용하는 특징이 있어요.
A9: 아이브는 데뷔 후 다양한 음악 활동과 공연 활동을 할 예정입니다.
A10: 아이브의 멤버는 태연, 보라, 장미, 디은, 저린 입니다.


In [118]:
# 25: 질문과 답변만 뽑자
bad_instruction_data={}
for x in bad_qa:

    q_s = x['instruction'].replace("\n\n","\n").split("\n")
    a_s = x['output'].replace("\n\n","\n").split("\n")

    # 한줄씩 자른 다음,  : 기준으로 잘라서 question과 answer만 뽑자!
    for qq, aa in zip(q_s, a_s):
        question = qq.split(":")[1].strip()
        answer = aa.split(":")[1].strip()
        bad_instruction_data[question] = answer # 추후에 작업 편의를 위해 질문을 Key로 하는 dict 형태로 저장

In [119]:
q1 = list(bad_instruction_data.keys())[0]
a1 = bad_instruction_data[q1]

print("질문:", q1)
print("답변:", a1)

질문: 아이브는 어떤 소속사에 소속되어 있나요?
답변: 아이브는 스타제국 에서 소속되어 있어요.


### 좋은 질의응답과 나쁜 질의응답을 만들었으니, 이제 reward 모델의 학습을 위한 형태로 변환해 봅시다

In [130]:
# 26: prompt, chosen, rejected 형태로 데이터 생성

reward_dataset=[]
for a in instruction_data:

    question = a['instruction']
    chosen = a['output'] # 좋은 답변

    if bad_instruction_data.get(question) is None:
        continue # API 에러로 인해 답변을 못받은 질문

    rejected = bad_instruction_data[question] # 나쁜 답변

    reward_dataset.append({'prompt': question,'chosen': chosen, 'rejected': rejected})

In [129]:
reward_dataset[0]

{'prompt': '아이브는 어떤 소속사에 소속되어 있나요?',
 'chosen': '아이브는 스타쉽엔터테인먼트에 소속되어 있습니다.',
 'rejected': '아이브는 스타제국 에서 소속되어 있어요.'}

In [ ]:
len(reward_dataset)

248

In [ ]:
# 28: 직접 만든 데이터와 합치기
reward_dataset.extend(hand_writen_data)

In [ ]:
# 29: 적절히 학습/평가 데이터로 나누기

train_rm = []
test_rm = []
for a in reward_dataset:
    if random.random() < 0.1:
        test_rm.append(a)
    else:
        train_rm.append(a)

In [ ]:
# 29: 저장(이름 바꿔주세요!!!!)

import json
with open("/content/drive/MyDrive/mygpt-lecture/data/stage2. RM/ive_train_rm.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in train_rm: file.write(json.dumps(i, ensure_ascii=False) + "\n")

with open("/content/drive/MyDrive/mygpt-lecture/data/stage2. RM/ive_test_rm.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in test_rm: file.write(json.dumps(i, ensure_ascii=False) + "\n")

# Stage 3 PPO 데이터 만들기, 질문만 있으면 됩니다!!

In [ ]:
# 30: 자신이 만든 사전학습 데이터 읽어오기!!

data=[]
for a in open("/content/drive/MyDrive/mygpt-lecture/data/stage1. domain_adaptive_pretraining/domain_adaptive_ive_pretrain.jsonl", 'r'):
    data.append(json.loads(a))

In [131]:
# 31: 최대한 다양한 질문을 만들어 낼 수 있는 프롬프트로 작성! 하나의 내용당 20개의 질문 요청

PROMPT="다음 주제와 내용을 보고 답변 할 수 있는 다양한 형태와 난이도의 창의적인 질문 20개를 만들어줘. n주제:{title}\n내용:{content}\n\
질문과 질문 사이는 반드시 ###을 붙여줘.\n\
답변의 형태는 다음과 같아.\n\
\n\
### Q1:질문1\n\
### Q2:질문2\n\
### Q3:질문3\n\
...\n\
### Q20:질문20\n\
"

In [132]:
# 32: 생성한 데이터를 저장할 리스트
question_only_data=[]
error_data=[]

In [ ]:
# 33: API Call

from tqdm import tqdm
for a in tqdm(data):

    title = a['instruction'].split("주제:")[-1]
    content = a['output']
    try:
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=[
                {'role':"system","content":"You are a helpful assistant."},
                {'role':"user","content":PROMPT.format_map({'title':title, 'content':content})},
                ],
            temperature=1 # 다양하게 만들기 위해 Temperature 1로 셋팅
        )
        question_only_data.append(response['choices'][0]['message']['content'])
    except Exception as e:
        error_data.append(a)
        print(title)
        print(e)

100%|██████████| 25/25 [09:16<00:00, 22.27s/it]


In [ ]:
question_only_data[-1]

'Q1: IZ*ONE 멤버 중 LOVE DIVE 활동을 기점으로 활동명이 변경된 멤버는 누구인가요?\nA1: 안유진과 장원영\n\nQ2: 혈액형이 모두 다른 멤버는 누구인가요?\nA2: 가을, 안유진, 레이, 리즈, 장원영, 이서\n\nQ3: 멤버 중 몇 명이 서울공연예술고등학교를 중퇴 또는 졸업한 적이 있는가요?\nA3: 3명 (안유진, 레이, 장원영)\n\nQ4: IZ*ONE이 대중음악계 영향력 있는 전문가 투표에서 걸그룹 부문에서 몇 위를 차지했나요?\nA4: 3위\n\nQ5: 모든 멤버가 코로나19 확진 판정을 받은 적이 있는가요?\nA5: 예\n\nQ6: 2023년 잼버리 축제에서 멤버들은 어떤 헤어스타일을 할까요?\nA6: 데뷔 때와 비슷한 헤어스타일\n\nQ7: IZ*ONE에는 외동인 멤버가 있나요?\nA7: 아니요, 없습니다.\n\nQ8: 안유진, 레이, 원영은 각각 언니와 누나, 오빠와 형제 자매인가요?\nA8: 유진, 원영은 언니, 레이는 누나입니다.\n\nQ9: 리즈와 이서는 각각 형제와 자매가 있나요?\nA9: 리즈는 남동생이 있고, 이서는 여동생이 있습니다.\n\nQ10: LOVE DIVE 활동 이전에는 모든 멤버가 어떤 종류의 활동명을 사용했나요?\nA10: 성씨를 제외한 본명 및 예명'

In [133]:
# 34: 데이터 전처리

prompt_dataset=[]
for q in question_only_data:
    q_s = q.replace("\n\n","\n").split("\n")

    # 한줄씩 자른 다음, : 기준으로 잘라서 question 뽑기!
    for qq in q_s:
        prompt_dataset.append({'prompt':qq.split(":")[1].lstrip()})

In [ ]:
len(prompt_dataset)

500

In [ ]:
for a in random.sample(prompt_dataset, 3):
    print(a)

{'prompt': '아이브의 멤버들이 가장 자주 하는 취미나 특기는 무엇일까요?'}
{'prompt': '이 그룹의 멤버들 중 가을과 다른 멤버들 사이에 어떤 차이점이 있나요?'}
{'prompt': '"After LIKE"가 Spotify 글로벌 차트에서 23위에 올랐다는 사실은 어떤 의미를 가지나요?'}


In [ ]:
# 35: Train/Test 분리

import random
train=[]
test=[]
for a in prompt_dataset:
    if random.random() < 0.1:
        test.append(a)
    else:
        train.append(a)

In [ ]:
# 36: 데이터 저장(이름 주의!!!)

import json
with open("/content/drive/MyDrive/mygpt-lecture/data/stage3. PPO/ive_train_ppo.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in train: file.write(json.dumps(i, ensure_ascii=False) + "\n")

with open("/content/drive/MyDrive/mygpt-lecture/data/stage3. PPO/ive_test_ppo.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in test: file.write(json.dumps(i, ensure_ascii=False) + "\n")